# Merge the Dataset

## Read the Adult, Exam, Lab Dataset

In [9]:
import pandas as pd
import numpy as np

adult = pd.read_csv("adult.csv")
adult.head()

,SEQN,DMARETHN,HSSEX,HSAGEIR,HFA8R,HFF18,HAC5A3,HAC5A4,HAD1,HAD5R,...,HAT21CD,HAT21MET,HAT22S,HAT23CD,HAT23MET,HAT24S,HAT25CD,HAT25MET,HAT26S,WTPFEX6
0,3,3,1,21,12,1,2.0,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1737.58
1,4,3,2,32,16,2,2.0,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1730.23
2,9,1,2,48,16,2,1.0,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19521.36
3,10,1,1,35,17,2,NaN,NaN,2,NaN,...,15430.0,10.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,28080.57
4,11,3,1,48,7,2,1.0,2.0,1,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1248.43


In [10]:
adult.shape

(20050, 52)

In [11]:
exam = pd.read_csv('exam.csv')
exam.head()

,SEQN,BMPWT,BMPBMI,BMPHT,BMPSITHT,PEPPREG,PEPPACE,PEP12A1,MAPF12,MAPF12R,HXPH2,HSSEX3,HSAGEIR3
0,3,81.60,25.5,178.8,93.1,2.0,2.0,468.0,NaN,NaN,NaN,1,21
1,4,61.65,23.4,162.2,86.1,2.0,2.0,751.0,NaN,2.0,NaN,2,32
2,7,13.05,13.8,97.3,51.3,NaN,NaN,NaN,NaN,NaN,NaN,2,3
3,9,68.05,27.6,156.9,84.9,2.0,2.0,513.0,2.0,2.0,NaN,2,48
4,10,92.50,29.4,177.4,93.7,2.0,2.0,455.0,NaN,NaN,NaN,1,35


In [12]:
exam.shape

(31311, 13)

In [13]:
lab = pd.read_csv('lab.csv')
lab.head()

,SEQN,G1P,G1PSI,G1PCODE,G1PTIM1,G1PTIM2,G2PSI,I1P,I1PSI,HSSEX2,HSAGEIR2
0,3,106.0,5.884,NaN,NaN,NaN,NaN,9.32,55.92,1,21
1,4,88.9,4.935,NaN,NaN,NaN,NaN,5.97,35.82,2,32
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
3,9,111.3,6.178,NaN,116.0,122.0,7.882,10.80,64.80,2,48
4,10,85.5,4.746,NaN,NaN,NaN,NaN,5.77,34.62,1,35


In [14]:
lab.shape

(29314, 11)

## Merge the three dataset by 'SEQN' (sequence number)

In [15]:
# merge the adult and exam dataframes
adult_exam = pd.merge(adult, exam, on='SEQN')
adult_exam.shape

(18162, 64)

In [16]:
# merge the adult_exam and lab dataframes
adult_exam_lab = pd.merge(adult_exam, lab, on='SEQN')
adult_exam_lab.shape

(18162, 74)

# Filter the dataset we want, Age: 40-74, Non-pregnant, Race, Insulin diabetes

## Age

In [17]:
# only keep age in 40-74 range, "HSAGEIR" is the age in years
adult_exam_lab = adult_exam_lab[adult_exam_lab['HSAGEIR'] <= 74]
adult_exam_lab = adult_exam_lab[adult_exam_lab['HSAGEIR'] >= 40]
adult_exam_lab.shape

(7948, 74)

## Pregnant

In [18]:
adult_exam_lab['MAPF12R'].value_counts()

MAPF12R
2.0    2337
8.0      17
1.0       5
9.0       1
Name: count, dtype: int64

In [19]:
# Pregnant women were excluded
adult_exam_lab = adult_exam_lab[adult_exam_lab['MAPF12R'] != 1]
adult_exam_lab.shape


(7943, 74)

## Race

In [20]:
adult_exam_lab["DMARETHN"].value_counts()

DMARETHN
1    3512
2    2135
3    1970
4     326
Name: count, dtype: int64

In [21]:
# exclude non-Hispanic whites, non-Hispanic blacks, and Mexican Americans, "DMARETHN" is the race/ethnicity
# 1: Non-Hispanic white, 2: non-Hispanic blacks, 3: Mexican Americans, and 4: other
# drop the rows which  "DMARETHN" is 4
adult_exam_lab = adult_exam_lab[adult_exam_lab["DMARETHN"] != 4]
adult_exam_lab.shape

(7617, 74)

## Diabetes

In [22]:
len(adult_exam_lab[adult_exam_lab["HAD5R"]<40])

151

In [23]:
# exclude 55 subjects with insulin-treated diabetes diagnosed before age 40 because they presumably had type 1 diabetes
# "HAD5R": How old were you when a doctor first told you that you had diabetes (not including during pregnancy)? (years)
# drop the rows which "HAD5R" is less than 40 and keep the null value
adult_exam_lab = adult_exam_lab[(adult_exam_lab["HAD5R"] > 40) | (adult_exam_lab["HAD5R"].isnull()) ]
adult_exam_lab.shape

(7424, 74)

In [24]:
# check for sex
# this would be deleted
adult_exam_lab['HSSEX'].value_counts()

HSSEX
2    3819
1    3605
Name: count, dtype: int64

# Output as csv file

In [25]:
adult_exam_lab.to_csv("adult_exam_lab.csv", index=False)